### setup before running

1. download all 2019 data from "https://webrobots.io/kickstarter-datasets/"
2. make a folder called "raw_data"
3. within "raw_data" make 3 sub-folders with the date of the pulls in the format MM-DD-YYYY
3. extract downloaded data into each of the 3 sub-folders appropriately
4. in line 4 below change the "path_master" variable to your "raw_data" link
5. run (~7min)

### merge all csv

In [2]:
import re
import os
import pandas as pd
path_master = 'raw_data/'
df_kickstarter = pd.DataFrame()

def add_date(path):
    return path[-10:-1]

directories = list()
for root, dirs, files in os.walk(path_master, topdown=False):
    for name in dirs:
        directories.append(os.path.join(root, name)+'/')

file_names = list()
for path, subdirs, files in os.walk(path_master):
    for filename in files:
        if filename not in file_names: file_names.append(filename)

for i in directories: 
    for j in file_names:
        try:
            with open(i+j, encoding='utf8') as fp:
                df_interim = pd.read_csv(fp, keep_default_na=False)
            df_interim['download_date'] = add_date(i)
            df_interim['download_date'] = pd.to_datetime(df_interim['download_date'], format='%m-%d-%Y')
            #global df_kickstarter
            df_kickstarter = pd.concat([df_kickstarter,df_interim])
            print('SUCCESS: '+i+j)
        except:
            print('FAIL: '+i+j)
            pass
df_kickstarter.to_csv(path_master+'df_kickstarter'+'.csv')

FAIL: raw_data/01-17-2019/.DS_Store
SUCCESS: raw_data/01-17-2019/Kickstarter040.csv
SUCCESS: raw_data/01-17-2019/Kickstarter054.csv
SUCCESS: raw_data/01-17-2019/Kickstarter041.csv
SUCCESS: raw_data/01-17-2019/Kickstarter043.csv
SUCCESS: raw_data/01-17-2019/Kickstarter042.csv
SUCCESS: raw_data/01-17-2019/Kickstarter052.csv
SUCCESS: raw_data/01-17-2019/Kickstarter046.csv
SUCCESS: raw_data/01-17-2019/Kickstarter.csv
SUCCESS: raw_data/01-17-2019/Kickstarter047.csv
SUCCESS: raw_data/01-17-2019/Kickstarter053.csv
SUCCESS: raw_data/01-17-2019/Kickstarter045.csv
SUCCESS: raw_data/01-17-2019/Kickstarter051.csv
SUCCESS: raw_data/01-17-2019/Kickstarter050.csv
SUCCESS: raw_data/01-17-2019/Kickstarter044.csv
SUCCESS: raw_data/01-17-2019/Kickstarter023.csv
SUCCESS: raw_data/01-17-2019/Kickstarter037.csv
SUCCESS: raw_data/01-17-2019/Kickstarter036.csv
SUCCESS: raw_data/01-17-2019/Kickstarter022.csv
SUCCESS: raw_data/01-17-2019/Kickstarter008.csv
SUCCESS: raw_data/01-17-2019/Kickstarter034.csv
SUCCESS

### read in random sample (10%) of full df_kickstarter to work with

In [128]:
import pandas as pd
import os
import random
filename = 'raw_data/df_kickstarter.csv'

p = 0.01  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)

6247

#Categorization

In [129]:
import numpy as np
import json

reserve "Cat_Name"(encoding), "backer_count", "blurb"(for NLP),"converted_pledged_amount","country"(encoding),"launched_at"(convert to time),"currency"(encoding),"deadline"(convert to time),"goal","pledged","staff_pick"(meaning?,encoding), "state"(encoding),"usd_pledged","name"(for NLP)

Don't reserve: "Location" is too noisy, too many 1s, "splotlight" is too correlated with result,"state_changed_at" almost equals to "deadline" 

In [130]:
cat_json = df['category'].apply(json.loads)
cat_name=list(map(lambda d: d['slug'], cat_json))
df['Cat_Name']=cat_name

In [131]:
#df=df.df.dropna(subset=["location"])
#loc_list=list(map(lambda d:json.loads(d)["name"], df['location']))

In [132]:
#df['Loc_Name']=loc_list

In [133]:
features_dict=dict() #inquiry for the actual value represented by the numerical value

In [134]:
for col in df.columns:
    if col in ['Cat_Name', 'country', 'currency', 'staff_pick','state']:
        df[col], indicators = pd.factorize(df[col])
        features_dict[col]=indicators

In [138]:
reserve=['Cat_Name', 'country', 'currency', 'staff_pick','state',"backers_count","blurb","converted_pledged_amount","goal","launched_at","pledged","usd_pledged","deadline","name"]

In [139]:
df=df[reserve]

In [140]:
df

,Cat_Name,country,currency,staff_pick,state,backers_count,blurb,converted_pledged_amount,goal,launched_at,pledged,usd_pledged,deadline,name
0,0,0,0,0,0,26,A supernatural horror short film about a disgr...,4308,2000.0,1479547850,4308.00,4308.000000,1480305600,The Harrowing - A Supernatural Horror Short
1,1,1,1,0,1,2,We perceive an opportunity to acquire signific...,5,1500000.0,1409112056,6.00,5.577126,1413000056,Helping Rural Farmers Earn More With Hydroponi...
2,2,0,0,0,1,1,"Save The Logs is a fun, creative way for all g...",10,3500.0,1409676690,10.00,10.000000,1412268690,Save The Logs
3,3,0,0,0,0,57,These power house sauces will elevate your foo...,12220,12000.0,1443554658,12220.00,12220.000000,1446353760,3 Ninjas hot sauce
4,4,0,0,0,0,29,The film is about a vampire hunter who wants r...,1920,1000.0,1362198951,1920.00,1920.000000,1362803751,"Please help support my project, ""The Cure""!"
5,5,0,0,0,1,0,This is a 2D game in progress called Vilande O...,0,5200.0,1488599854,0.00,0.000000,1491188254,Vilande Online MMO RPG
6,6,0,0,1,0,4133,Matt is dancing around the world again to show...,146075,125000.0,1447802252,146075.97,146075.970000,1450394252,Where the Heck is Matt?
7,7,0,0,0,0,2234,A 1-4 player board game with solo/cooperative ...,332310,70000.0,1539705669,332310.00,332310.000000,1540915269,Wild Assent - Survive the Arena and Tame the W...
8,8,2,2,0,1,1,Meet Panda Water.\nA drink and a water bottle;...,97,1200.0,1471296714,74.00,95.617118,1473888714,"Panda Water, bamboo good enough for pandas!"
9,9,0,0,0,0,30,Support this campaign to have artist Jason Rob...,6280,6000.0,1509645671,6280.00,6280.000000,1512241271,Houston Astros World Series Painting


In [137]:
features_dict["country"][9]

'DE'

In [142]:
#df['Loc_Name'].value_counts() #should we combine or delete the rows with few category appearance

In [143]:
#df.isnull().values.any()

In [144]:
#df['current_currency'].unique()